In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

from pyspark.sql.types import *
from pyspark.sql.functions import *
import urllib

# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

aws_user = '0a1153066525'


In [0]:
suffix='pin'
stream_name = 'streaming-'+ aws_user + '-' + suffix
df_pin = spark \
.readStream \
.format('kinesis') \
.option('streamName',stream_name) \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

suffix='geo'
stream_name = 'streaming-'+ aws_user + '-' + suffix
df_geo = spark \
.readStream \
.format('kinesis') \
.option('streamName',stream_name) \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()

suffix='user'
stream_name = 'streaming-'+ aws_user + '-' + suffix
df_user = spark \
.readStream \
.format('kinesis') \
.option('streamName',stream_name) \
.option('initialPosition','earliest') \
.option('region','us-east-1') \
.option('awsAccessKey', ACCESS_KEY) \
.option('awsSecretKey', SECRET_KEY) \
.load()


In [0]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, StringType
import PinterestTransformations

In [0]:
# clean the df_pin DataFrame
# Define the schema for parsing the JSON
# original raw data structure before transformation
raw_pin_columns = {
    "category",
    "description", 
    "downloaded", 
    "follower_count",
    "image_src", 
    "index", 
    "is_image_or_video", 
    "poster_name", 
    "save_location", 
    "tag_list",
    "title", 
    "unique_id"
}
# could be improved in refactoring by having a list of string types and long types, then build the schema and extract the columns so column names do not need to be repeated in code
raw_pin_json_schema = StructType([
    StructField("category", StringType(), True),
    StructField("description", StringType(), True),
    StructField("downloaded", LongType(), True),
    StructField("follower_count", StringType(), True),
    StructField("image_src", StringType(), True),
    StructField("index", LongType(), True),
    StructField("is_image_or_video", StringType(), True),
    StructField("poster_name", StringType(), True),
    StructField("save_location", StringType(), True),
    StructField("tag_list", StringType(), True),
    StructField("title", StringType(), True),
    StructField("unique_id", StringType(), True)
])

# Extract specific fields from the parsed JSON
# could be improved in refactoring by moving the rest of this to a function since it is the same for every stream
df_pin = df_pin.selectExpr("CAST(data as STRING)")
df_pin = df_pin.withColumn("parsed_json", from_json(df_pin.data, raw_pin_json_schema))
for column in raw_pin_columns:
    df_pin = df_pin.withColumn(column, df_pin.parsed_json[column])
df_pin = df_pin.drop("data","parsed_json")

df_pin = PinterestTransformations.clean_pin(df_pin)
# display(df_pin)

In [0]:
# clean the df_geo DataFrame
# Define the schema for parsing the JSON
# original raw data structure before transformation
raw_geo_columns = {
    "country",
    "ind", 
    "latitude", 
    "longitude",
    "timestamp"
}
raw_geo_json_schema = StructType([
    StructField("country", StringType(), True),
    StructField("ind", LongType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("timestamp", StringType(), True)
])

# Extract specific fields from the parsed JSON
df_geo = df_geo.selectExpr("CAST(data as STRING)")
df_geo = df_geo.withColumn("parsed_json", from_json(df_geo.data, raw_geo_json_schema))
for column in raw_geo_columns:
    df_geo = df_geo.withColumn(column, df_geo.parsed_json[column])
df_geo = df_geo.drop("data","parsed_json")

df_geo = PinterestTransformations.clean_geo(df_geo)
# display(df_geo)

In [0]:
# clean the df_user DataFrame
# Define the schema for parsing the JSON
# original raw data structure before transformation
raw_user_columns = {
    "age",
    "date_joined", 
    "first_name", 
    "ind",
    "last_name"
}
raw_user_json_schema = StructType([
    StructField("age", LongType(), True),
    StructField("date_joined", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("ind", LongType(), True),
    StructField("last_name", StringType(), True)
])

# Extract specific fields from the parsed JSON
df_user = df_user.selectExpr("CAST(data as STRING)")
df_user = df_user.withColumn("parsed_json", from_json(df_user.data, raw_user_json_schema))
for column in raw_user_columns:
    df_user = df_user.withColumn(column, df_user.parsed_json[column])
df_user = df_user.drop("data","parsed_json")

df_user = PinterestTransformations.clean_user(df_user)
# display(df_user)

In [0]:
# write data into DataBricks.

df_pin.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
  .table(aws_user + "_pin_table")

df_geo.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
  .table(aws_user + "_geo_table")

df_user.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", "/tmp/kinesis/_checkpoints/") \
  .table(aws_user + "_user_table")
